In [6]:
! lamin load scprint

💡 found cached instance metadata: /home/ml4ig1/.lamin/instance--jkobject--scprint.env
❗ 

Your database is not up to date with your installed Python library.

Your database has the latest migrations:
['lnschema_bionty.0022_rename_datasets_cellline_collections_and_more']

Your Python library has the latest migrations:
['lnschema_bionty.0023_rename_publicsource_encode_uid']

Only if you are an admin and manage migrations manually, deploy them to the database: lamin migrate deploy

Otherwise, downgrade your Python library to match the database!

💡 loaded instance: jkobject/scprint


In [2]:
import lamindb as ln
import lnschema_bionty as lb

import pandas as pd
import scanpy as sc 

from lightning.pytorch import Trainer, seed_everything
seed_everything(42, workers=True)

from scprint import scPrint
from scprint.utils import getBiomartTable

from scdataloader import Dataset
from scdataloader import DataLoader


%load_ext autoreload
%autoreload 2

SystemError: initialization of _internal failed without raising an exception

In [3]:
## Gene embeddings
organism = lb.Organism.filter(ontology_id="NCBITaxon:9606").one()
genedf = lb.Gene.filter(organism_id=organism.id).df()
embeddings = embed(genedf=genedf,
    organism="homo_sapiens",
    cache=True,
    fasta_path="/tmp/data/fasta/",
    embedding_size=1024,)
embeddings.to_parquet('../../data/temp/embeddings.parquet')
embeddings = pd.read_parquet('../../data/temp/embeddings.parquet')

# and annotations
biomart = getBiomartTable(attributes=['start_position', 'chromosome_name']).set_index('ensembl_gene_id')
genedf = genedf.set_index('ensembl_gene_id')

genedf = genedf.loc[~genedf.index.duplicated(keep='first')]
biomart = biomart.loc[~biomart.index.duplicated(keep='first')]

genedf = genedf.join(biomart[['start_position', 'chromosome_name']], how='inner').sort_values(by=['chromosome_name', 'start_position'])

# and location
c = []
i = 0
prev_position = -100000
prev_chromosome = None
for _, r in genedf.iterrows():
    if r['chromosome_name'] != prev_chromosome or r['start_position'] - prev_position > 10_000:
        i += 1
    c.append(i)
    prev_position = r['start_position']
    prev_chromosome = r['chromosome_name']
print(f'reduced the size to {len(set(c))/len(genedf)}')
genedf['group'] = c

In [4]:
# OR directly load the dataset
name="preprocessed dataset"
dataset = ln.Dataset.filter(name=name).one()
dataset.artifacts.count()

NameError: name 'ln' is not defined

In [ ]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_labels = [
    "cell_type_ontology_term_id",
    #"tissue_ontology_term_id",
    "disease_ontology_term_id",
    #"development_stage_ontology_term_id",
    "assay_ontology_term_id",
    'self_reported_ethnicity_ontology_term_id',

]

labels_weighted_sampling = hierarchical_labels+[
    'sex_ontology_term_id',
]

all_labels = labels_weighted_sampling+[
    #'dataset_id',
    #'cell_culture',
    "dpt_group",
    "heat_diff",
    "nnz",
    "total_counts"
]

In [ ]:
mdataset = Dataset(dataset, genedf, gene_embedding=embeddings, organisms=['"NCBITaxon:9606"'], obs=all_labels, encode_obs=labels_weighted_sampling, map_hierarchy=hierarchical_labels, )
print(mdataset)
dataloader = BaseDataLoader(mdataset, label_to_weight=labels_weighted_sampling, batch_size=4, num_workers=1)
len(dataloader)

In [ ]:
import numpy as np


In [ ]:
MAX_SIZE=2000
ADD_ZEROS=200
# get the unseen info and don't add any unseen
# get the I most expressed genes, add randomly some unexpressed genes that are not unseen
loc = np.argsort(i[0],1)[:,-(MAX_SIZE):] 
random_indices = []

for j in range(i[0].shape[0]):
    zero_locs = np.where(i[0][j] == 0)[0]
    random_indices.append(zero_locs[np.random.choice(len(zero_locs), ADD_ZEROS, replace=False)])
random_indices = np.array(random_indices)
loc = np.concatenate((random_indices, loc), axis=1)

In [ ]:
model = scPrint(
    gene_df = ,
    d_model = 64,
    nhead = 2,
    d_hid = 64,
    nlayers = 2,
    layers_cls = [],
    labels = {},
    cls_hierarchy = {},
    dropout= 0.2,
    transformer_backend = "fast",
    use_precomputed_gene_embbeddings = True,
    do_gene_pos_enc = True,
    do_mvc = True,
    do_adv = True,
)

In [ ]:
# sets seeds for numpy, torch and python.random.
trainer = Trainer(deterministic=True)